# Salty Boussinesq Melting

This notebook will perform the analysis of the salty Boussinesq melting simulations.

# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as de

d = de.operators.differentiate
integ = de.operators.integrate

import glob
from dedalus.tools import post
import file_tools as flt
import interpolation as ip
import time
import os
import logging
import sys
logger = logging.getLogger(__name__)

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Computer Modern Roman']})
rc('text', usetex=True)

# Coordinate remapping


In this notebook, we use a coordinate remapping to simulate flow underneath an ice sheet with topography.

The coordinate system is defined as follows.

Cartesian coordinates are $t, x, z$.

Remapped coordinates and $\tau, \xi, \zeta$, which are defined as

\begin{align}
\tau &= \frac{t}{\mathcal{T}} &
\xi &= \frac{x}{\mathcal{L}} &
\zeta &= \frac{z}{\mathcal{L}\eta(\tau,\xi)}
\end{align}

This lets us write all the partial derivatives with respect to each other

\begin{align}
\frac{\partial {\xi}_i}{\partial{x}_j} &= 
    \begin{bmatrix}
    \partial_t \tau & \partial_t \xi & \partial_t \zeta \\
    \partial_x \tau & \partial_x \xi & \partial_x \zeta \\
    \partial_z \tau & \partial_z \xi & \partial_z \zeta \\
    \end{bmatrix}
    = \begin{bmatrix}
    \frac{1}{\mathcal{T}} & 0 & -\frac{1}{\mathcal{T}} \zeta \frac{\partial_\tau \eta}{\eta}\\
    0 & \frac{1}{\mathcal{L}} & -\frac{1}{\mathcal{L}} \zeta \frac{\partial_\xi \eta}{\eta}\\
    0 & 0 & \frac{1}{\mathcal{L} \eta}
    \end{bmatrix}\\
\frac{\partial {x}_i}{\partial{\xi}_j} &= 
    \begin{bmatrix}
    \partial_\tau t & \partial_\tau x & \partial_\tau z \\
    \partial_\xi t & \partial_\xi x & \partial_\xi z \\
    \partial_\zeta t & \partial_\zeta x & \partial_\zeta z \\
    \end{bmatrix} 
    = \begin{bmatrix}
    \mathcal{T} & 0 & \mathcal{L} \zeta \partial_\tau \eta\\
    0 & \mathcal{L} & \mathcal{L} \zeta \partial_\xi \eta\\
    0 & 0 & \mathcal{L} \eta
    \end{bmatrix}
\end{align}

We will determine the differential geometric quantities in the spatial directions, and consider time derivatives separately (as a metric makes sense spatially only).
Remember, everything here will in general also vary in time

We get the **tangent vectors**

\begin{align}
e_1 = \frac{\partial\bf{x}}{\partial\xi} \cong \frac{\partial}{\partial\xi} &= \mathcal{L} \hat{e}_1 + \zeta \partial_\xi \eta  \, \mathcal{L}\hat{e}_2\\
e_2 = \frac{\partial\bf{x}}{\partial\zeta} \cong \frac{\partial}{\partial\zeta} &=\eta \, \mathcal{L} \hat{e}_2
\end{align}

The have corresponding **dual/covectors**

\begin{align}
\omega^1 = \nabla \xi = \bf{d} \xi &= \frac{1}{\mathcal{L}}\hat{\omega}^1 \\
\omega^2 = \nabla \zeta = \bf{d} \zeta &= - \zeta \frac{\partial_\xi \eta}{\eta} \, \frac{1}{\mathcal{L}} \hat{\omega}^1 +  \frac{1}{\eta}\frac{1}{\mathcal{L}} \hat{\omega}^2
\end{align}

We can then calculate the **metric** components

\begin{align}
g_{11} &= \mathcal{L}^2(1 + \zeta^2 \partial_\xi \eta^2)  &
    g^{11} &= \frac{1}{\mathcal{L}^2}\\
g_{12} = g_{21} &= \mathcal{L}^2 \zeta \eta \partial_\xi \eta &
    g^{12} = g^{21} &= - \frac{1}{\mathcal{L}^2} \zeta \frac{\partial_\xi \eta}{\eta} \\
g_{22} &= \mathcal{L}^2 \eta^2 &
    g^{22} &= \frac{1}{\mathcal{L}^2} \frac{1}{\eta^2}\left(1 + \zeta^2 \partial_\xi \eta^2 \right)
\end{align}

This has determinant of
$$ g = \mathcal{L}^4 \eta^2 $$
and **Jacobian**
$$ \sqrt{g} = \mathcal{L}^2 \eta$$

We calculate the **connection coefficients**/Christoffel symbols directly using
\begin{align}
\Gamma^\gamma_{\alpha \beta} = \langle \omega^\gamma, \nabla_\beta e_\alpha\rangle
\end{align}

Hence
\begin{align}
\Gamma^1_{11} &= 0 & \Gamma^2_{11} &= \zeta \frac{\partial^2_\xi \eta}{\eta}\\
\Gamma^1_{12} &= 0 & \Gamma^2_{12} &= \frac{\partial_\xi \eta}{\eta}\\
\Gamma^1_{21} &= 0 & \Gamma^2_{21} &= \frac{\partial_\xi \eta}{\eta}\\
\Gamma^1_{22} &= 0 & \Gamma^2_{22} &= 0
\end{align}
Of course, for coordinate bases we have symmetry in the last two components.

We can write the **divergence** using
$$\nabla \cdot u = \nabla u : I = (\omega^j \nabla_j)(u^i e_i): = (u^i_{,j} + u^j \Gamma^i_{ji})\omega^j\cdot e_i = (u^i_{,i} + u^j \Gamma^i_{ji})$$
$$\nabla \cdot u = u^i_{;i} = u^i_{,i} + \Gamma^i_{j i} u^j = u^1_{,1} + u^2_{,2} + \frac{\partial_\xi \eta}{\eta} u^1$$

We write the **vorticity** (in covariant components) as

$$q = \nabla \times u = \mathcal{E}:\nabla u = (\mathcal{E}^{ij}e_i e_j):(\omega^k \nabla_k)(u^l \, e_l) = \frac{1}{\sqrt{g}} [ij] u^l_{;k}(e_i\cdot\omega^k)(e_j\cdot e_l)$$
$$ = \frac{1}{\sqrt{g}} [ij] g_{jl} u^l_{;i} = \frac{1}{\sqrt{g}} ( g_{2l} u^l_{;1} - g_{1l} u^l_{;2})$$

We can write the **curl of the vorticity** as 

$$ \nabla \times q = \mathcal{E}\cdot\nabla q = (\mathcal{E}^{ij}e_i e_j)\cdot(\omega^k \nabla_k)q =  \frac{1}{\sqrt{g}}[ij]q_{,i} e_j = \frac{1}{\sqrt{g}}(q_{,1}e_2 - q_{,2}e_1) $$

Note that this sign appears to be incorrect.

The **gradient** of the pressure is just

$$\nabla p = g^{ji} p_{,j} e_i$$

The **cross product** $u \times q$ is 

$$u\times q = \mathcal{E}\cdot u q = (\mathcal{E}_{ij}\omega^i\omega^j)\cdot(q u^k e_k) = \sqrt{g}q [ij] \delta^i_k u^k \omega^j = \sqrt{g} g^{jk} [ij] u^i e_k$$
$$ = \sqrt{g}q(g^{2k} u^1 - g^{1k}u^2)e_k$$

**Time derivative**

$$ \partial_t = \partial_t\tau \partial_\tau + \partial_t \xi \partial_\xi + \partial_t \zeta \partial_\zeta = \frac{1}{\mathcal{T}} (\partial_\tau - \zeta \frac{\partial_\tau \eta}{\eta} \partial_\zeta)$$

And 

$$ \partial_t u = \partial_t (u^i e_i) = \partial_t u^i e_i + u^i \partial_t e_i$$

where

$$\partial_\tau e_1 = \mathcal{L} \zeta \partial_\tau \partial_\xi \eta \hat{e}_2 = \frac{\zeta}{\eta}\partial_\tau\partial_\xi \eta\, e_2$$
$$\partial_\tau e_2 = \mathcal{L} \partial_\tau \eta \hat{e}_2 = \frac{\partial_\tau\eta}{\eta}\, e_2$$

## Plotting the coordinate transformation

In [ ]:
# Create Cartesian grid
n = 64
xb = de.Fourier('x',n,interval=(0,4))
zb = de.Chebyshev('z',n,interval=(0,1))
d0 = de.Domain([xb,zb],grid_dtype=np.float64)
xg,zg = d0.grids()
xxg,zzg = xg+0*zg, 0*xg+zg

# Create remapped grid
ξb = de.Fourier('ξ',n,interval=(0,4))
ζb = de.Chebyshev('ζ',n,interval=(0,1))
d1 = de.Domain([ξb,ζb],grid_dtype=np.float64)
ξg,ζg = d1.grids()
ξξg,ζζg = ξg+0*ζg, 0*ξg+ζg
ηg = 1 + .25*(1-np.cos(np.pi*xg/2))

# Create dedalus fields to store various quantities for each domain
f0,f1,f2 = {},{},{}
def create_fields(names,dic,d,dim='x'): 
    for name in names: 
        dic[name] = d.new_field()
        dic[name].meta[dim]['parity'] = 1
create_fields(['x','z','η','ξ','ζ1','ζ2'],f0,d0)
create_fields(['x','z1','z2','η','ξ','ζ'],f1,d1,dim='ξ')
# Quantities on Cartesian grid
f0['x']['g'] = xg
f0['z']['g'] = zg
f0['η']['g'] = ηg
f0['ξ']['g'] = xg
f0['ζ1']['g'] = (zg-ηg)/(2-ηg)
f0['ζ2']['g'] = zg/ηg
# Quantities in remapped space
f1['η']['g'] = ηg
f1['ξ']['g'] = ξg
f1['ζ']['g'] = ζg
f1['x']['g'] = xg
f1['z1']['g'] = ηg + (2-ηg)*ζg
f1['z2']['g'] = ηg*ζg

In [ ]:
# Plot ξ and ζ level sets in Cartesian (x,z) space (at ξ,ζ grid points)
fig, ax = plt.subplots(figsize=(8,4),constrained_layout=True)
cplot1 = ax.contourf(f1['x']['g'],f1['z1']['g'],f1['ζ']['g'],np.linspace(0,1,6),cmap='viridis_r',vmin=0,vmax=1)
cplot2 = ax.contourf(f1['x']['g'],f1['z2']['g'],f1['ζ']['g'],np.linspace(0,1,6),cmap='magma',vmin=-.2,vmax=1)
ax.contour(f1['x']['g'],f1['z1']['g'],f1['ξ']['g'],np.linspace(0,4,17),linewidths=0.5,colors='k',)
ax.contour(f1['x']['g'],f1['z2']['g'],f1['ξ']['g'],np.linspace(0,4,17),linewidths=0.5,colors='k',)
ax.contour(f1['x']['g'],f1['z1']['g'],f1['ζ']['g'],np.linspace(0,1,6),linewidths=0.5,colors='k',)
ax.contour(f1['x']['g'],f1['z2']['g'],f1['ζ']['g'],np.linspace(0,1,6),linewidths=0.5,colors='k',)
ax.plot(ξg,ηg,'k')
for spine in ax.spines: ax.spines[spine].set_visible(False)
ax.set_xlabel('$x$',fontsize=16)
ax.set_ylabel('$z$',fontsize=16)
ax.set_title('$\\zeta^-$ and $\\zeta^+$ in Cartesian coordinates',fontsize=16)
ax.set(aspect=1,xticks=np.linspace(0,4,16,endpoint=False),yticks=np.linspace(0,2,9))
cbax1 = fig.add_axes([0.91, 0.54, 0.03, .385]) 
cbax2 = fig.add_axes([0.91, 0.125, 0.03, .385]) 
cbar1 = fig.colorbar(cplot1,cax=cbax1,shrink=.5,panchor=(0,.75))
cbar2 = fig.colorbar(cplot2,cax=cbax2,shrink=.5,anchor=(0,.25))
cbar1.ax.set_ylabel('$\\zeta^-$',rotation=0,fontsize=15,labelpad=20)
cbar2.ax.set_ylabel('$\\zeta^+$',rotation=0,fontsize=15,labelpad=20)
cbar1.outline.set_visible(False)
cbar2.outline.set_visible(False)
plt.savefig('zeta-cartesian-both.pdf',bbox_inches='tight')